In [39]:
import pandas as pd
import numpy as np
import datetime

In [40]:
# TV_data is only local and allocatd national data into the 186 dmas
# for the purpose of national tv data, use another file
TV_data=pd.read_csv("/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/TV/finaltvlogs_nationaldistributed.csv",dtype=str)
TV_data['Act Impression']=TV_data['Act Impression'].astype(float)
TV_data['Net Cost']=TV_data['Net Cost'].astype(float)
TV_data['week_start_date']=TV_data['Week BL'].apply(lambda x: datetime.datetime.strptime(x,"%m/%d/%y").date())

In [41]:
TV_data=TV_data[['week_start_date','cleaned DMA','Network','Media Type','Act Impression','Net Cost']]
TV_data=TV_data.rename(columns={'cleaned DMA':'cleaned dma','Network':'placement','Media Type':'submedia','Act Impression':'impression','Net Cost':'cost'})
TV_data['submedia']=TV_data['submedia'].replace(['National Cable','DirecTV'],"National")
TV_data['submedia']=TV_data['submedia'].replace(['Spot Cable', 'Spot Broadcast', 'FOOTPRINT-SINCLAIR'],"Local")
# TV_data['week_start_date']=TV_data['week_start_date'].apply(lambda x:datetime.datetime.strptime(x,"%Y-%m-%d").date())
TV_data['week_end_date']=TV_data['week_start_date']+datetime.timedelta(days=6)
TV_data['click']=0
TV_data['media']='TV'
TV_data=TV_data[['week_start_date','week_end_date','cleaned dma','media','submedia','placement','impression','click','cost']]

In [42]:
# Confirmed with Spencer, becaudse of the national TV has been allocated to dma level and labled as "National" in submedia
TV_data.reset_index(inplace=True)
del TV_data['index']
TV_data=TV_data[TV_data['cleaned dma']!="National"]

In [43]:
Digital_1=pd.read_csv("/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/replaced_cricular_20180226.csv",dtype=str)
Digital_2=pd.read_csv("/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/Criteo_TMR_Weekly.csv",dtype=str)
Digital_1=Digital_1[Digital_1['media']!="TV"]
Digital_2['cleaned dma']=Digital_2['cleaned dma'].replace('National','xx')
Digital_1['submedia']=np.where(Digital_1['media']=='Radio','Radio',Digital_1['submedia'])

In [44]:
Digital_1=Digital_1[~pd.isnull(Digital_1['week date'])]
Digital_1['week date']=Digital_1['week date'].apply(lambda x: datetime.datetime.strptime(x,"%m/%d/%Y").date())
Digital_1=Digital_1.rename(columns={'week date':'week_start_date'})
Digital_1['week_end_date']=Digital_1['week_start_date']+datetime.timedelta(days=6)
Digital_1=Digital_1[['week_start_date','week_end_date','cleaned dma','media','submedia','placement','impression','click','cost']]

In [45]:
Digital_2['week_start_date']=Digital_2['week_start_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
Digital_2['week_end_date']=Digital_2['week_end_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
Digital=Digital_1.append(Digital_2)
Digital['impression']=Digital['impression'].astype(float)
Digital['click']=Digital['click'].astype(float)
Digital['cost']=Digital['cost'].astype(float)

In [46]:
Valassis_CPM=pd.read_csv("/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/Valassis_CPM.csv",dtype=str)
Valassis_CPM['CPM']=Valassis_CPM['Total CPM'].str.replace("$","")
Valassis_CPM['CPM']=Valassis_CPM['CPM'].astype(float)
Valassis_CPM['week_start_date']=Valassis_CPM['Event\nDate'].apply(lambda x: datetime.datetime.strptime(x,"%m/%d/%y").date())
Valassis_CPM=Valassis_CPM[['week_start_date','CPM']]
Valassis_TMR=Digital[Digital['submedia']=="Valassis"]
Valassis_TMR=pd.merge(Valassis_TMR,Valassis_CPM,on="week_start_date",how='left')
del Valassis_TMR['cost']
Valassis_TMR['cost']=Valassis_TMR['impression']*Valassis_TMR['CPM']/1000
Valassis_TMR=Valassis_TMR[Digital.columns]

In [47]:
Digital=Digital[Digital['submedia']!="Valassis"]
Digital=Digital.append(Valassis_TMR)
Digital=Digital.sort_values(['week_start_date','cleaned dma','media','submedia','placement'])

In [48]:
TMR_Data_DMA=Digital.append(TV_data)
TMR_Data_DMA.reset_index(inplace=True)
del (TMR_Data_DMA['index'])
TMR_Data_DMA.to_csv("/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/output/TMR_Data_DMA.csv",index=False)

In [49]:
Digital_National=Digital.groupby(['week_start_date','week_end_date','media','submedia','placement'])['impression','click','cost'].sum()
Digital_National.reset_index(inplace=True)

In [50]:
TV_National=pd.read_csv("/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/TV/finaltvlogs_0306.csv",dtype=str)
TV_National['week_start_date']=TV_National['Week BL'].apply(lambda x:datetime.datetime.strptime(x,"%m/%d/%y").date())

TV_National=TV_National[['week_start_date','Media Type','Network','Act Impression','Net Cost']]
TV_National=TV_National.rename(columns={'Media Type':'submedia','Network':'placement','Act Impression':'impression','Net Cost':'cost'})
TV_National['submedia']=TV_National['submedia'].replace(['National Cable','DirecTV'],"National")
TV_National['submedia']=TV_National['submedia'].replace(['Spot Cable', 'Spot Broadcast', 'FOOTPRINT-SINCLAIR'],"Local")
TV_National['week_end_date']=TV_National['week_start_date']+datetime.timedelta(days=6)
TV_National['click']=0
TV_National['media']='TV'
TV_National=TV_National[['week_start_date','week_end_date','media','submedia','placement','impression','click','cost']]
TV_National['impression']=TV_National['impression'].astype(float)
TV_National['cost']=TV_National['cost'].astype(float)
TV_National=TV_National.groupby(['week_start_date','week_end_date','media','submedia','placement'])['impression','click','cost'].sum().reset_index()


In [51]:
TMR_Data_National=Digital_National.append(TV_National)
TMR_Data_National.reset_index(inplace=True)
del (TMR_Data_National['index'])
TMR_Data_National.to_csv("/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/output/TMR_Data_National.csv",index=False)

# Long to wide

## National

In [52]:
Sales=pd.read_csv("/Users/Jian/Desktop/Media Storm/Big Lots/Ongoing/Datorama/Output_2018-02-17/combined_sales_long_2018-02-17.csv")
del Sales['week_indicator']
DMA=pd.read_excel("/Users/Jian/Desktop/Media Storm/Big Lots/TMR/Reformat non TV/all_store_DMA.xlsx")
DMA=DMA[['location_id','cleaned dma']]
Sales=pd.merge(Sales,DMA,on='location_id',how='left')
Sales['week_end_date']=Sales['week_end_date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').date())
Sales['week_start_date']=Sales['week_end_date']-datetime.timedelta(days=6)
Sales=Sales[(Sales['week_end_date']>=datetime.datetime.strptime('2016-10-03',"%Y-%m-%d").date()) &(Sales['week_end_date']<=datetime.datetime.strptime('2017-12-31',"%Y-%m-%d").date())]

In [53]:
Sales_May6=Sales[(Sales['week_end_date']==datetime.datetime.strptime('2017-05-13',"%Y-%m-%d").date()) | (Sales['week_end_date']==datetime.datetime.strptime('2017-04-29',"%Y-%m-%d").date())]

In [54]:
Sales_May6=Sales_May6.groupby(['location_id'])['transactions'].mean().to_frame()
Sales_May6.reset_index(inplace=True)

In [55]:
Sales_May6=Sales_May6.rename(columns={'transactions':'average_trans'})
Sales_May6['week_end_date']=datetime.datetime.strptime("2017-05-06","%Y-%m-%d").date()

In [56]:
Sales=pd.merge(Sales,Sales_May6,on=['location_id','week_end_date'],how='left')

In [57]:
Sales['transactions']=np.where(Sales['week_end_date']==datetime.datetime.strptime("2017-05-06","%Y-%m-%d").date(),
                               Sales['average_trans'],
                               Sales['transactions'])
del Sales['average_trans']

In [58]:
Sales_DMA=Sales.groupby(['week_start_date','week_end_date','cleaned dma'])['sales','transactions'].sum().fillna(0).reset_index()
# Because some stores with traffic but no sales, and the DMA is missing as "xx"
if (Sales_DMA[Sales_DMA['cleaned dma']=="xx"]['sales'].unique()==0) & (Sales_DMA[Sales_DMA['cleaned dma']=="xx"]['transactions'].unique()==0):
    Sales_DMA=Sales_DMA[Sales_DMA['cleaned dma']!="xx"]
len(Sales_DMA['cleaned dma'].unique())

186

In [59]:
Competitor=pd.read_csv("/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/Competitors_Counts.csv",dtype=str)
Competitor=Competitor[['cleaned dma','Competitor']]
Competitor['Competitor']=Competitor['Competitor'].astype(float)

In [60]:
Store_count=pd.read_csv("/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/store_counts_by_DMA.csv",dtype=str)
Store_count['week_start_date']=Store_count['week_start_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
Store_count['store_count']=Store_count['store_count'].astype(int)

In [61]:
# Add store cout and competitor to DMA level sales
Sales_DMA=pd.merge(Sales_DMA,Store_count,on=['cleaned dma','week_start_date'],how='left')
Sales_DMA=pd.merge(Sales_DMA,Competitor,on=['cleaned dma'],how='left')

In [62]:
Ecommerce=pd.read_csv("/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/Ecommerce_Sales.csv",dtype=str)
Ecommerce['week_start_date']=Ecommerce['week_start_date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').date())
Ecommerce['week_end_date']=Ecommerce['week_end_date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').date())
Ecommerce['sales:Ecommerce']=Ecommerce['sales:Ecommerce'].astype(float)
Ecommerce['sales:Ecommerce']=Ecommerce['transactions:Ecommerce'].astype(float)

In [63]:
# Add ecommerce and competitor to national level sales
Sales_National=Sales.groupby(['week_start_date','week_end_date'])['sales','transactions'].sum().fillna(0).reset_index()
Store_count_National=Store_count.groupby(['week_start_date'])['store_count'].sum().to_frame()
Store_count_National.reset_index(inplace=True)
Sales_National=pd.merge(Sales_National,Store_count_National,on=['week_start_date'],how='left')
Sales_National=pd.merge(Sales_National,Ecommerce,on=['week_start_date','week_end_date'],how='left')

In [64]:
# Submedia level

TMR_Data_National_submedia=TMR_Data_National[['week_start_date','week_end_date','media','submedia','placement','impression','click','cost']]

TMR_Data_National_submedia=TMR_Data_National_submedia.groupby(['week_start_date','week_end_date','media','submedia'])['impression','click','cost'].apply(sum)
TMR_Data_National_submedia.reset_index(inplace=True)
TMR_Data_National_submedia['media_submedia']=TMR_Data_National_submedia['media']+"_"+TMR_Data_National_submedia['submedia']
del TMR_Data_National_submedia['media']
del TMR_Data_National_submedia['submedia']

national_submedia_wide=TMR_Data_National_submedia.pivot_table(index=['week_start_date','week_end_date'],columns='media_submedia',values=['impression','click','cost']).fillna(0)
new_columns=[]
for i in range(len(national_submedia_wide.columns)):
    x=national_submedia_wide.columns[i]
    y=str(x[0]+":"+x[1])
    new_columns.append(y)
national_submedia_wide.columns=new_columns
national_submedia_wide.reset_index(inplace=True)
national_submedia_wide['week_start_date']=national_submedia_wide['week_start_date'].apply(lambda x: x.date())
national_submedia_wide['week_end_date']=national_submedia_wide['week_end_date'].apply(lambda x: x.date())
national_submedia_wide=pd.merge(Sales_National,national_submedia_wide,on=['week_start_date','week_end_date'],how='left')


In [65]:
# Placement level
TMR_Data_National_placement=TMR_Data_National[['week_start_date','week_end_date','media','submedia','placement','impression','click','cost']]
TMR_Data_National_placement=TMR_Data_National_placement.groupby(['week_start_date','week_end_date','media','submedia','placement'])['impression','click','cost'].apply(sum)
TMR_Data_National_placement.reset_index(inplace=True)
TMR_Data_National_placement['media_submedia_placement']=TMR_Data_National_placement['media']+"_"+TMR_Data_National_placement['submedia']+"_"+TMR_Data_National_placement['placement']
del TMR_Data_National_placement['media']
del TMR_Data_National_placement['submedia']
del TMR_Data_National_placement['placement']
national_placement_wide=TMR_Data_National_placement.pivot_table(index=['week_start_date','week_end_date'],columns='media_submedia_placement',values=['impression','click','cost']).fillna(0)
new_columns=[]
for i in range(len(national_placement_wide.columns)):
    x=national_placement_wide.columns[i]
    y=str(x[0]+":"+x[1])
    new_columns.append(y)
national_placement_wide.columns=new_columns
national_placement_wide.reset_index(inplace=True)
national_placement_wide['week_start_date']=national_placement_wide['week_start_date'].apply(lambda x: x.date())
national_placement_wide['week_end_date']=national_placement_wide['week_end_date'].apply(lambda x: x.date())
national_placement_wide=pd.merge(Sales_National,national_placement_wide,on=['week_start_date','week_end_date'],how='left')

## DMA level

In [66]:
TMR_Data_DMA_submedia=TMR_Data_DMA[['week_start_date','week_end_date','cleaned dma','media','submedia','placement','impression','click','cost']]

TMR_Data_DMA_submedia=TMR_Data_DMA_submedia.groupby(['week_start_date','week_end_date','media','submedia','cleaned dma'])['impression','click','cost'].apply(sum)
TMR_Data_DMA_submedia.reset_index(inplace=True)
TMR_Data_DMA_submedia['media_submedia']=TMR_Data_DMA_submedia['media']+"_"+TMR_Data_DMA_submedia['submedia']
del TMR_Data_DMA_submedia['media']
del TMR_Data_DMA_submedia['submedia']

DMA_submedia_wide=TMR_Data_DMA_submedia.pivot_table(index=['week_start_date','week_end_date','cleaned dma'],columns='media_submedia',values=['impression','click','cost']).fillna(0)
new_columns=[]
for i in range(len(DMA_submedia_wide.columns)):
    x=DMA_submedia_wide.columns[i]
    y=str(x[0]+":"+x[1])
    new_columns.append(y)
DMA_submedia_wide.columns=new_columns
DMA_submedia_wide.reset_index(inplace=True)
DMA_submedia_wide['week_start_date']=DMA_submedia_wide['week_start_date'].apply(lambda x: x.date())
DMA_submedia_wide['week_end_date']=DMA_submedia_wide['week_end_date'].apply(lambda x: x.date())
DMA_submedia_wide=pd.merge(Sales_DMA,DMA_submedia_wide,on=['week_start_date','week_end_date','cleaned dma'],how='left')

In [67]:
TMR_Data_DMA_placement=TMR_Data_DMA[['week_start_date','week_end_date','cleaned dma','media','submedia','placement','impression','click','cost']]
TMR_Data_DMA_placement=TMR_Data_DMA_placement.groupby(['week_start_date','week_end_date','cleaned dma','media','submedia','placement'])['impression','click','cost'].apply(sum)
TMR_Data_DMA_placement.reset_index(inplace=True)
TMR_Data_DMA_placement['media_submedia_placement']=TMR_Data_DMA_placement['media']+"_"+TMR_Data_DMA_placement['submedia']+"_"+TMR_Data_DMA_placement['placement']
del TMR_Data_DMA_placement['media']
del TMR_Data_DMA_placement['submedia']
del TMR_Data_DMA_placement['placement']
DMA_placement_wide=TMR_Data_DMA_placement.pivot_table(index=['week_start_date','week_end_date','cleaned dma'],columns='media_submedia_placement',values=['impression','click','cost']).fillna(0)
new_columns=[]
for i in range(len(DMA_placement_wide.columns)):
    x=DMA_placement_wide.columns[i]
    y=str(x[0]+":"+x[1])
    new_columns.append(y)
DMA_placement_wide.columns=new_columns
DMA_placement_wide.reset_index(inplace=True)
DMA_placement_wide['week_start_date']=DMA_placement_wide['week_start_date'].apply(lambda x: x.date())
DMA_placement_wide['week_end_date']=DMA_placement_wide['week_end_date'].apply(lambda x: x.date())
DMA_placement_wide=pd.merge(Sales_DMA,DMA_placement_wide,on=['week_start_date','week_end_date','cleaned dma'],how='left')

In [68]:
DMA_submedia_wide.to_csv('/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/output/DMA_submedia_wide.csv',index=False)
DMA_placement_wide.to_csv('/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/output/DMA_placement_wide.csv',index=False)

national_submedia_wide.to_csv('/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/output/national_submedia_wide.csv',index=False)
national_placement_wide.to_csv('/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/output/national_placement_wide.csv',index=False)

In [69]:
DMA_12=pd.read_csv("/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/12DMAs_Group.csv",dtype=str)

In [70]:
# 12 DMA and balance
DMA_submedia_wide_12=DMA_submedia_wide[DMA_submedia_wide['cleaned dma'].isin(DMA_12['cleaned dma'])]
DMA_submedia_wide_184=DMA_submedia_wide[~DMA_submedia_wide['cleaned dma'].isin(DMA_12['cleaned dma'])]

DMA_submedia_wide_12.to_csv("/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/output/Common12_DMA_submedia.csv")
DMA_submedia_wide_184.to_csv("/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/output/Balance_of_12_DMA_submedia.csv")

In [71]:
Agg_DMA_submedia_wide_12=DMA_submedia_wide_12.groupby(['week_start_date','week_end_date','cleaned dma']).sum()
Agg_DMA_submedia_wide_184=DMA_submedia_wide_184.groupby(['week_start_date','week_end_date','cleaned dma']).sum()

Agg_DMA_submedia_wide_12.to_csv("/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/output/Agg_DMA_submedia_wide_12.csv")
Agg_DMA_submedia_wide_184.to_csv("/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/output/Agg_DMA_submedia_wide_184.csv")

# To Bruce

In [72]:
writer=pd.ExcelWriter('/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/output/Bruce/BL_TMR wide format_JL_20180306.xlsx', engine='xlsxwriter')

DMA_92=pd.read_excel("/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/Holiday_DMAs_Group.xlsx",dtype=str,sheetname="Holiday_92")
DMA_56=pd.read_excel("/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/Holiday_DMAs_Group.xlsx",dtype=str,sheetname="Holiday_56")
DMA_36=pd.read_excel("/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/Holiday_DMAs_Group.xlsx",dtype=str,sheetname="Holiday_36")

Sales_DMA_LastYear=Sales_DMA[(Sales_DMA['week_end_date']>=datetime.datetime.strptime("2016-11-05","%Y-%m-%d").date()) &
                   (Sales_DMA['week_end_date']<=datetime.datetime.strptime("2016-12-31","%Y-%m-%d").date())]

Sales_DMA_ThisYear=Sales_DMA[(Sales_DMA['week_end_date']>=datetime.datetime.strptime("2017-11-04","%Y-%m-%d").date()) &
                   (Sales_DMA['week_end_date']<=datetime.datetime.strptime("2017-12-30","%Y-%m-%d").date())]
del Sales_DMA_LastYear['store_count']
del Sales_DMA_LastYear['Competitor']
Sales_DMA_LastYear['week_start_date']=Sales_DMA_LastYear['week_start_date'].apply(lambda x:x+datetime.timedelta(days=364))
Sales_DMA_LastYear['week_end_date']=Sales_DMA_LastYear['week_end_date'].apply(lambda x:x+datetime.timedelta(days=364))
Sales_DMA_LastYear=Sales_DMA_LastYear.rename(columns={'sales':'sales_ly','transactions':'transactions_ly'})


del Sales_DMA_ThisYear['store_count']
del Sales_DMA_ThisYear['Competitor']
Sales_DMA_ThisYear=pd.merge(Sales_DMA_ThisYear,Sales_DMA_LastYear,on=['week_start_date','week_end_date','cleaned dma'],how='outer')

Sales_DMA_ThisYear['sales_incremental']=Sales_DMA_ThisYear['sales']-Sales_DMA_ThisYear['sales_ly']
Sales_DMA_ThisYear['transactions_incremental']=Sales_DMA_ThisYear['transactions']-Sales_DMA_ThisYear['transactions_ly']
del Sales_DMA_ThisYear['sales_ly']
del Sales_DMA_ThisYear['transactions_ly']

Sales_DMA_Holiday=Sales_DMA_ThisYear[['week_start_date','week_end_date','cleaned dma','sales','transactions']]
Sales_DMA_Holiday_92=Sales_DMA_ThisYear[Sales_DMA_ThisYear['cleaned dma'].isin(DMA_92['cleaned dma'])]
Sales_DMA_Holiday_36=Sales_DMA_ThisYear[Sales_DMA_ThisYear['cleaned dma'].isin(DMA_36['cleaned dma'])]
Sales_DMA_Holiday_56=Sales_DMA_ThisYear[Sales_DMA_ThisYear['cleaned dma'].isin(DMA_56['cleaned dma'])]
Sales_DMA_Holiday_94=Sales_DMA_ThisYear[~Sales_DMA_ThisYear['cleaned dma'].isin(DMA_92['cleaned dma'])]


/Users/Jian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/Jian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [73]:
TMR_Data_DMA_media=TMR_Data_DMA[['week_start_date','week_end_date','cleaned dma','media','submedia','impression','click','cost']]
'''
for i in range(len(TMR_Data_DMA_media)):
    if (TMR_Data_DMA_media['media'][i]=="TV") & (TMR_Data_DMA_media['submedia'][i]=="Local"):
        TMR_Data_DMA_media['media'][i]="TV_Local"
    if (TMR_Data_DMA_media['media'][i]=="TV") & (TMR_Data_DMA_media['submedia'][i]=="National"):
        TMR_Data_DMA_media['media'][i]="TV_National"
'''
TMR_Data_DMA_media['media']=np.where((TMR_Data_DMA_media['media']=="TV")& (TMR_Data_DMA_media['submedia']=="Local"),"TV_Local",
                                    np.where((TMR_Data_DMA_media['media']=="TV")& (TMR_Data_DMA_media['submedia']=="National"),"TV_National",
                                            TMR_Data_DMA_media['media']))
        
        
TMR_Data_DMA_media=TMR_Data_DMA_media[['week_start_date','week_end_date','cleaned dma','media','impression','click','cost']]
TMR_Data_DMA_media=TMR_Data_DMA_media.groupby(['week_start_date','week_end_date','cleaned dma','media'])['impression','click','cost'].sum()
TMR_Data_DMA_media.reset_index(inplace=True)

TMR_Data_DMA_media_92=TMR_Data_DMA_media[TMR_Data_DMA_media['cleaned dma'].isin(DMA_92['cleaned dma'])]
TMR_Data_DMA_media_56=TMR_Data_DMA_media[TMR_Data_DMA_media['cleaned dma'].isin(DMA_56['cleaned dma'])]
TMR_Data_DMA_media_36=TMR_Data_DMA_media[TMR_Data_DMA_media['cleaned dma'].isin(DMA_36['cleaned dma'])]
TMR_Data_DMA_media_94=TMR_Data_DMA_media[~TMR_Data_DMA_media['cleaned dma'].isin(DMA_92['cleaned dma'])]


/Users/Jian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [74]:
TMR_Data_DMA_media_92=TMR_Data_DMA_media_92.pivot_table(index=['week_start_date','week_end_date','cleaned dma'],columns='media',values=['impression','click','cost']).fillna(0)
TMR_Data_DMA_media_92.reset_index(inplace=True)
Sales_TMR_Data_DMA_media_92=pd.merge(Sales_DMA_Holiday_92,TMR_Data_DMA_media_92,on=['week_start_date','week_end_date','cleaned dma'],how='left')
Sales_TMR_Data_DMA_media_92.to_excel(writer,"Holiday_92_DMAs",index=False)

TMR_Data_DMA_media_56=TMR_Data_DMA_media_56.pivot_table(index=['week_start_date','week_end_date','cleaned dma'],columns='media',values=['impression','click','cost']).fillna(0)
TMR_Data_DMA_media_56.reset_index(inplace=True)
Sales_TMR_Data_DMA_media_56=pd.merge(Sales_DMA_Holiday_56,TMR_Data_DMA_media_56,on=['week_start_date','week_end_date','cleaned dma'],how='left')
Sales_TMR_Data_DMA_media_56.to_excel(writer,"Holiday_56_TV_DMAs",index=False)

TMR_Data_DMA_media_36=TMR_Data_DMA_media_36.pivot_table(index=['week_start_date','week_end_date','cleaned dma'],columns='media',values=['impression','click','cost']).fillna(0)
TMR_Data_DMA_media_36.reset_index(inplace=True)
Sales_TMR_Data_DMA_media_36=pd.merge(Sales_DMA_Holiday_36,TMR_Data_DMA_media_36,on=['week_start_date','week_end_date','cleaned dma'],how='left')
Sales_TMR_Data_DMA_media_36.to_excel(writer,"Holiday_36_Digital_DMAs",index=False)

TMR_Data_DMA_media_94=TMR_Data_DMA_media_94.pivot_table(index=['week_start_date','week_end_date','cleaned dma'],columns='media',values=['impression','click','cost']).fillna(0)
TMR_Data_DMA_media_94.reset_index(inplace=True)
Sales_TMR_Data_DMA_media_94=pd.merge(Sales_DMA_Holiday_94,TMR_Data_DMA_media_94,on=['week_start_date','week_end_date','cleaned dma'],how='left')
Sales_TMR_Data_DMA_media_94.to_excel(writer,"Holiday_94_Balance_DMAs",index=False)
writer.save()

/Users/Jian/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:551: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/Jian/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:862: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  self.right = self.right.drop(right_drop, axis=1)


In [29]:
# Check Missing DMA impression in Digital
import pandas as pd
import datetime
TMR=pd.read_csv("/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/output/TMR_Data_DMA.csv")
TMR['week_end_date']=TMR['week_end_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
TMR=TMR[TMR['week_end_date']>datetime.datetime.strptime("2017-11-02","%Y-%m-%d").date()]
TMR=TMR[TMR['week_end_date']<datetime.datetime.strptime("2017-12-31","%Y-%m-%d").date()]
TMR.to_csv("/Users/Jian/Desktop/Media Storm/Big Lots/TMR/TMR Long/output/TMR_Data_DMA_Holiday_QC.csv")